# Intensivtag KI: Hands-On Cats & Dogs
### Nachvollziehbarkeit und die Grenzen ''künstlicher Intelligenz''

In diesem Praxisteil wollen wir anhand eines Modells das Katzen und Hunde voneinander unterscheiden kann die Nachvollziehbarkeit und die Grenzen von KI-Modellen untersuchen.

<br /><br />
### 1) Importe

Software ist nie perfekt. Das gilt auch für <i>Tensorflow</i>, die derzeit führenden Software-Bibliothek für künstliche neuronale Netzwerke, die von Google entwickelt wird. <i>Tensorflow</i> wirft häufig irreführende Fehlermeldungen aus, auch wenn prinzipiell alles OK ist. Der folgende Code dient dazu, diese zu unterbinden.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings("ignore")

import tensorflow
tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

Wir importieren jetzt die Bibliotheken, die wir tatsächlich benötigen, sowie einige Hilfsfunktionen, die wir in die Datei <i>cats_and_dogs_utils.py</i> ausgelagert haben. Bei Interesse könnt ihr gerne diese Datei öffnen, um den entsprechenden Code zu sehen.

In [ ]:
import pickle
import matplotlib.pyplot

from cats_and_dogs_utils import construct_model, predict, process_image, attention, capture_image, perfect_animal

%matplotlib inline

### 2) Das Modell

Das Modell, das wir hier untersuchen, ist ein Convolutional Neural Network ('faltendes neuronales Netzwerk') ähnlich zu dem, das ihr bei der Ziffererkennung kennengelernt habt. Es bekommt als Eingabe 150 x 150 Pixel große farbige Bilder und gibt als Ausgabe eine Wahrscheinlichkeit, dass das Bild einen Hund zeigt, an. Wenn diese Wahrscheinlichkeit geringer als 50% ist, interpretieren wir das Bild als eine Katze.

Da wir es mit größeren und mehrfarbigen Bildern, sowie einem komplexeren Modell zu tun haben, dauert das Training des Netzwerk deutlich länger, es benötigt mehrere Stunden. Darum überspringen wir das Training und nutzen das fertig trainierte Modell.

Wir laden nun die Modellarchitektur sowie die trainierten Modellparameter und zeigen die Modellarchitektur an.

In [ ]:
model = construct_model()
model.load_weights('data/cats_and_dogs.h5')
model.summary()

<br><br>
Auch wenn wir das Training überspringen, ist es interessant, sich die Trainingshistorie anschauen, die wir aus einer abgespeicherten Datei laden und plotten.

In [ ]:
with open('data/history_200.pickle', 'rb') as file:
    training_history = pickle.load(file)

acc = training_history['acc']
val_acc = training_history['val_acc']
loss = training_history['loss']
val_loss = training_history['val_loss']

matplotlib.pyplot.plot(val_acc, color='green', label='Genauigkeit Validierungsdaten')
matplotlib.pyplot.plot(acc, color='blue', label='Genauigkeit Trainingsdaten')
matplotlib.pyplot.xlabel('Trainingsepoche')
matplotlib.pyplot.legend()

matplotlib.pyplot.figure()
matplotlib.pyplot.plot(val_loss, color='green', label='Kosten Validierungsdaten')
matplotlib.pyplot.plot(loss, color='blue', label='Kosten Trainingsdaten')
matplotlib.pyplot.xlabel('Trainingsepoche')
matplotlib.pyplot.legend()
matplotlib.pyplot.show()

Das Modell wurde offenbar 200 Epochen lang trainiert und weist am Ende eine Genauigkeit von ca. 90% auf.

Auffällig ist, dass die Kurven für die Validerungsdaten sehr stark schwanken. Das liegt daran, dass der Validierungsdatensatz nur aus jeweils 1500 Hunde- und Katzenbildern besteht. Bei dieser vergleichsweise kleinen Datenmenge ist es statistisch sehr wahrscheinlich, dass die Genauigkeit sichtbar besser oder schlechter als der entsprechende Mittelwert des Modells ist.

Es fällt ebenfalls auf, dass (im Mittel) kein Overfitting stattfindet. Das liegt unter anderem daran, dass der Trainingsdatensatz, der aus jeweils 10000 Hunde- und Katzenbildern besteht, durch zufällige Drehungen, Verschiebungen, Spiegelungen und Verzerrungen künstlich vergrößert wurde. Eine genügend große Trainingsdatenmenge ist eine Möglichkeit, Overfitting zu vermeiden.

<br /><br />
### 3) Nachvollziehbarkeit und Grenzen der Modellvorhersage

Bei komplexen Modellen wie neuronalen Netzwerken ist es oft schwierig, genau nachzuvollziehen, wie das Modell zu einer Vorhersage gelangt, beziehungsweise was es beim Training 'fachlich' tatsächlich gelernt hat. Das ist je nach Anwendungsgebiet natürlich höchst problematisch.

Eine vollständige 'Black Box' sind die Modelle aber nicht und es gibt doch einige Möglichkeiten, ihnen 'unter die Haube zu schauen'.

Wir überprüfen zunächst die Modellvorhersage auf Bildern von drei mehr oder weniger bekannten Tieren aus dem Internet. Ihr könnt gerne weitere Bilder ausprobieren!

In [ ]:
# Url des Bildes
grumpy_cat_url = 'https://i.kinja-img.com/gawker-media/image/upload/s--UYWnBrHt--/c_scale,f_auto,fl_progressive,q_80,w_800/wmpvownqus8xwvylswsr.jpg'

# Umwandeln des Bildes in ein Format, das das Modell verarbeiten kann
grumpy_cat_image = process_image(grumpy_cat_url)

# Modellvorhersage
predict(model, grumpy_cat_image)

In [ ]:
grumpy_dog_url = 'https://images.fineartamerica.com/images-medium-large-5/grumpy-bulldog-john-daniels.jpg'
grumpy_dog_image = process_image(grumpy_dog_url)
predict(model, grumpy_dog_image)

In [ ]:
husky_url = 'https://cmeimg-a.akamaihd.net/640/photos.demandstudios.com/getty/article/211/219/104286802.jpg'
husky_image = process_image(husky_url)
predict(model, husky_image)

Bei den ersten beiden Bildern liegt das Modell mit großer Sicherheit richtig, beim letzten total daneben. Warum?

Um das nachvollziehen, müssen wir verstehen, was für ein 'Konzept' von Hunden und Katzen das Modell gelernt hat. Eine mögliche Frage, die wir uns stellen können: Wie sieht für das Modell eine perfekte Katze / ein perfekter Hund aus?

Es ist möglich künstliche Bilder zu erzeugen, bei denen das Modell sozusagen 'voll anschlägt'. Um eine künstliche 'perfekte Katze' zu erzeugen nimmt man ein Startbild und 'mutiert' es so lange, bis der 'Katzenscore' des Modells maximal wird. Das Ergebnis, mit Rauschen als Startbild, sieht so aus:

In [ ]:
perfect_cat = perfect_animal(model, 'cat')
matplotlib.pyplot.imshow(perfect_cat)
matplotlib.pyplot.axis('off')
matplotlib.pyplot.show()

Ein perfekter Hund sieht dagegen so aus:

In [ ]:
perfect_dog = perfect_animal(model, 'dog')
matplotlib.pyplot.imshow(perfect_dog)
matplotlib.pyplot.axis('off')
matplotlib.pyplot.show()

Aus diesen Bildern können wir die Hypothese ableiten, dass das Modell bei Katzen im wesentlichen auf Augen mit einem Schlitz als Pupille achtet, während es bei Hunden eher auf runde Augen oder die Schnauze achtet.

Wir können diese Hypothese mit einer anderen Methode überprüfen. Mit einem Verfahren, dessen Details wir aus Zeitgründen hier auslassen, ist es möglich, die Bereiche eines Bildes hervorzuheben, die für das Modell besonders 'katzenartig' oder 'hundeartig' sind. Es sind gewissermaßen die Bereiche, auf die das Modell besonders aufmerksam schaut.

Bei den folgenden Bildern ist zu beachten, dass es nicht so ist, dass das hellere Bild 'gewinnt'!

In [ ]:
attention(model, grumpy_dog_image)

In [ ]:
attention(model, grumpy_cat_image)

In [ ]:
attention(model, husky_image)

Die obigen Bilder passen sehr gut zu unserer Hypothese. Insbesondere hat das Modell offenbar den Strich in der Färbung der Stirn des Huskys als Katzenauge fehlinterpretiert daraus die finale Vorhersage abgeleitet, obwohl es die Hundeaugen und -schnauze auch erkannt hat.

Mit diesem Verständnis können wir nun aus einer Katze einen Hund machen und umgekehrt! Dazu überlagern wir einfach stufenweise das Originalbild mit einem 'perfekten' Bild, das aus dem Originalbild mit der oben beschrieben Methode entsteht. Ihr könnt gerne mit den Überlagerungsstufen experimentieren!

In [ ]:
start_image_cat = grumpy_cat_image
perfect_dog_2 = perfect_animal(model, 'dog', seed_input=start_image_cat)

weights = [0.9, 0.8, 0.5]
fig, axs = matplotlib.pyplot.subplots(1, len(weights), figsize=(len(weights) * 5, 5))
for i, weight in enumerate(weights):
    predict(model, start_image_cat * weight + perfect_dog_2 * (1 - weight), axs[i])

In [ ]:
start_image_dog = grumpy_dog_image
perfect_cat_2 = perfect_animal(model, 'cat', seed_input=start_image_dog)

weights = [0.9, 0.8, 0.5]
fig, axs = matplotlib.pyplot.subplots(1, len(weights), figsize=(len(weights) * 5, 5))
for i, weight in enumerate(weights):
    predict(model, start_image_dog * weight + perfect_cat_2 * (1 - weight), axs[i])

<br /><br />
### 4) Was bist du? :-)

Inzwischen ist deutlich geworden, dass das Modell klare Grenzen hat. Eine davon ist natürlich die Tatsache, dass es jedes Bild entweder als Katze oder als Hund interpretiert.

Das kannst du jetzt an dir selbst ausprobieren!

Mit dem folgenden Code nimmt deine Webcam ein Bild von dir auf. Wenn du den Code ausführst, wird dein Browser dich einmalig fragen, ob du den Zugriff auf deine Webcam erlaubst. Du kannst den Code nach Belieben ausführen um neue Bilder aufzunehmen.

In [ ]:
me_captured = capture_image()
me_captured

Mit deinem Bild kannst du jetzt alle obigen Schritte durchführen!

In [ ]:
# Wenn hier eine Fehlermeldung auftaucht, dann warst du zu schnell. :-)
# Die Verarbeitung des Webcam-Bildes kann ein paar Sekunden dauern.
# Führe die Zelle dann einfach nochmals aus.

me_image = process_image(me_captured)
predict(model, me_image)

In [ ]:
attention(model, me_image)

In [ ]:
perfect_dog_3 = perfect_animal(model, 'dog', seed_input=me_image)

weights = [0.9, 0.8, 0.5]
fig, axs = matplotlib.pyplot.subplots(1, len(weights), figsize=(len(weights) * 5, 5))
for i, weight in enumerate(weights):
    predict(model, me_image * weight + perfect_dog_3 * (1 - weight), axs[i])

In [ ]:
perfect_cat_3 = perfect_animal(model, 'cat', seed_input=me_image)

weights = [0.9, 0.8, 0.5]
fig, axs = matplotlib.pyplot.subplots(1, len(weights), figsize=(len(weights) * 5, 5))
for i, weight in enumerate(weights):
    predict(model, me_image * weight + perfect_cat_3 * (1 - weight), axs[i])